In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import udf, col, concat_ws, collect_list, lit, log
import sys
import re

spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

In [3]:
comment = "/Users/ting/Documents/CMPT733/Project/SOCC/raw/gnm_comments.csv"
df = spark.read.load(comment,
                     format='com.databricks.spark.csv', 
                     header='true', 
                     inferSchema='true')
df.head()

Row(article_id=10012655, comment_counter='source1_10012655_0', comment_author='modoglobe', timestamp=1364221200000.0, post_time=None, comment_text="I think the program needs some work and probably some very costly oversight to prevent human rights abuses according to this report from the Canadian Labour Congress:'Canada's Temporary Foreign Worker Program is far from being a modelinitiative. Given the experiences of the Canadian labour movement, it isabundantly clear that the program's design permits the exploitation ofmigrant workers. It operates to serve employers' interests with littlemeaningful regard for compliance, monitoring, or enforcement of nationalor subnational labour standards.'www.canadianlabour.ca/sites/default/files/pdfs/model-program-or-mistake-2011-en.pdf", TotalVotes='0.0', posVotes=None, negVotes=None, vote='none', reactions=None, replies=None, comment_id='33750cc126314c84b4babae99e97b347', parentID=None, threadID=None, streamId='10012655.0', edited='False', isModera

In [4]:
df_sub = df.select('article_id','comment_id','timestamp', 'TotalVotes','posVotes','negVotes','comment_text')

In [5]:
df_sub.dtypes

[('article_id', 'int'),
 ('comment_id', 'string'),
 ('timestamp', 'double'),
 ('TotalVotes', 'string'),
 ('posVotes', 'string'),
 ('negVotes', 'string'),
 ('comment_text', 'string')]

In [7]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

p_stemmer = PorterStemmer()

def review_to_wordlist( review, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
#     words = [p_stemmer.stem(w) for w in words]
    #
    # 5. Return a list of words
#     return(words)
    return( " ".join(words ))

def string2int(inputstring):
    if not inputstring:
        return 0.0
    return (float(inputstring))

udf_review = udf(lambda w: review_to_wordlist(w), StringType())
udf_str2int = udf(lambda s: string2int(s), FloatType())
# df_sub.withColumn("TotalVotes",udf_str2int(df_sub.TotalVotes)).show()
# df_sub.withColumn("posVotes",udf_str2int(df_sub.posVotes)).show()
df_sub.withColumn("negVotes",udf_str2int(df_sub.negVotes))

DataFrame[article_id: int, comment_id: string, timestamp: double, TotalVotes: string, posVotes: string, negVotes: float, comment_text: string]

In [11]:
# myDF=df_sub.withColumn("topic_word_count",topicWord(df_sub.comment_text))
clean_df = df_sub.select('article_id', 'comment_id', \
                         udf_review(df_sub.comment_text).alias('cleaned_text')) 

# clean_df.coalesce(1).write.csv('cleaned_comment', header = True)
clean_df.show()

+----------+--------------------+--------------------+
|article_id|          comment_id|        cleaned_text|
+----------+--------------------+--------------------+
|  10012655|33750cc126314c84b...|think program nee...|
|  10012655|4ce8f60b0ddd442c8...|offshoring revers...|
|  10012655|75900e8bd92c45149...|spell exploitatio...|
|  10012655|ac49765f024640ae9...|tfws place econom...|
|  10012655|c5b63fd3000e43069...|temporary workers...|
|  10012655|8817325c040b4974b...|sense playing gam...|
|  10012655|88861ede36de4818b...|never reason excu...|
|  10012655|ac87455db20f460f8...|tfws place econom...|
|  10012655|7444480408c24db69...|tfws work tenets ...|
|  10012655|400856b5f4e84ba18...|    like give thumbs|
|  10012655|0da3fd203ac14f1eb...|tfws proper place...|
|  10012655|82373e13cfa440b7a...|think hd mining t...|
|  10012655|022f42de0a5741bba...|thing developing ...|
|  10012655|f86888b1aedd4352a...|yep along way fin...|
|  10012655|82d38afae57c4ce49...|            wow fool|
|  1001265

In [55]:
# import operator 
# def word_count(row):
#     r_texts = re.sub(r'[^\w]', ' ', row['cleaned_text'])
#     words = r_texts.split(' ')
#     for w in words:
# #         yield comment_row(w, 1)
#         yield comment_row(row['article_id'], row['comment_id'], w, 1)

# comment_file = clean_df.select('article_id', 'comment_id', 'cleaned_text')

# comment_row = Row('article_id', 'comment_id', 'word', 'count')
# # comment_row = Row('word', 'count')
# comment_wordcount = comment_file.rdd.flatMap(word_count).toDF()

# comment_wordcount.createOrReplaceTempView('comment_wordcount')
# comment_wordcounts = spark.sql("""
#     SELECT article_id, comment_id, word, sum(count) as count
#     FROM comment_wordcount
#     GROUP BY article_id, comment_id, word
#     ORDER BY article_id, count DESC
# """)

# comment_wordcounts.show()
# # text_file.take(2)

+----------+--------------------+-------------+-----+
|article_id|          comment_id|         word|count|
+----------+--------------------+-------------+-----+
|    535381|bca183ec85d04ba9a...|          the|    5|
|    535381|bca183ec85d04ba9a...|       police|    4|
|    535381|bca183ec85d04ba9a...|           in|    4|
|    535381|bca183ec85d04ba9a...|        world|    3|
|    535381|bca183ec85d04ba9a...|         best|    3|
|    535381|bca183ec85d04ba9a...|       forces|    2|
|    535381|bca183ec85d04ba9a...|     canadian|    1|
|    535381|bca183ec85d04ba9a...|        force|    1|
|    535381|bca183ec85d04ba9a...|         list|    1|
|    535381|bca183ec85d04ba9a...|          com|    1|
|    535381|bca183ec85d04ba9a...|           it|    1|
|    535381|bca183ec85d04ba9a...|         http|    1|
|    535381|bca183ec85d04ba9a...|infotainworld|    1|
|    535381|bca183ec85d04ba9a...|       indeed|    1|
|    535381|bca183ec85d04ba9a...|           of|    1|
|    535381|bca183ec85d04ba9

In [56]:
# comment_wordcounts = spark.sql("""
#     SELECT article_id, comment_id, word, sum(count) as count
#     FROM comment_wordcount
#     GROUP BY article_id, comment_id, word
#     ORDER BY count DESC
# """)

# comment_wordcounts.show()

+----------+----------+----+-----+
|article_id|comment_id|word|count|
+----------+----------+----+-----+
|  33185073|  678094_0| the|  276|
|  33413032|  816447_0| the|  243|
|  33303579|  529466_0| the|  239|
|  33413032|  816447_0|that|  177|
|  33413032|  816447_0|  is|  176|
|  33185073|  678094_0| and|  168|
|  33413032|  816447_0|  of|  161|
|  33413032|  816447_0|   a|  155|
|  33413032|  816447_0| you|  153|
|  33413032|  816447_0|  to|  148|
|  33424030|  606482_0| the|  144|
|  33185073|  678094_0|  of|  132|
|  33123104|  568439_0| the|  124|
|  33303579|  529466_0|  to|  124|
|  33347006|  784316_0| the|  124|
|  33185073|  678094_0|  to|  124|
|  20259976|      null| the|  121|
|  18297978|  978845_0| the|  119|
|  33413032|  816447_0|   i|  117|
|  33185073|  678094_0|   a|  110|
+----------+----------+----+-----+
only showing top 20 rows



## Train Model - Bag of Words

In [12]:
import pandas as pd
import numpy as np
label_data = '/Users/ting/Documents/CMPT733/Project/sentimentAnalysis/kaggle/data/labeledTrainData.tsv'
train = pd.read_csv(label_data, header=0, \
                    delimiter="\t", quoting=3)
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [13]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_wordlist( train["review"][i] ) )

/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/ting/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [62]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

In [63]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit( train_data_features, train["sentiment"] )

In [69]:
test_data = '/Users/ting/Documents/CMPT733/Project/sentimentAnalysis/cleaned_comment/part-00000-803122b2-81f6-4428-8fc6-41962b3705d0-c000.csv'
test = pd.read_csv(test_data,quoting=3,error_bad_lines=False)
test.head()

b'Skipping line 197535: expected 3 fields, saw 25\n'


,article_id,comment_id,cleaned_text
0,10012655,33750cc126314c84b4babae99e97b347,think program needs work probably costly overs...
1,10012655,4ce8f60b0ddd442c8a3ac70c15feb954,offshoring reverse well union busting say good...
2,10012655,75900e8bd92c451491729551878a166d,spell exploitation disgusting practice sanctio...
3,10012655,ac49765f024640ae93e0913cdfbb4d48,tfws place economy canadians would leave home ...
4,10012655,c5b63fd3000e4306960411384e2999b2,temporary workers get paid tim horton rest com...


In [70]:
clean_test_reviews = test["cleaned_text"]
test_data_features = vectorizer.transform(clean_test_reviews.values.astype('U'))
# test_data_features = test_data_features.toarray()

In [71]:
test_data_features = test_data_features.toarray()
predition = forest.predict(test_data_features)

predition = np.asarray(predition)
test['sentiment'] = predition
test.to_csv('comment_w_sentiment_bw.csv')

In [153]:
predition = np.asarray(predition)
test['sentiment'] = predition
test.to_csv('comment_w_sentiment.csv')

## Train Model - Word Vectors

In [18]:
import pandas as pd
import numpy as np
label_data = '/Users/ting/Documents/CMPT733/Project/sentimentAnalysis/kaggle/data/labeledTrainData.tsv'
train = pd.read_csv(label_data, header=0, \
                    delimiter="\t", quoting=3)

unlabel_data = '/Users/ting/Documents/CMPT733/Project/sentimentAnalysis/kaggle/data/unlabeledTrainData.tsv'
unlabeled_train = pd.read_csv(unlabel_data, header=0, \
                                delimiter="\t", quoting=3 )

train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [19]:
test_data = "/Users/ting/Documents/CMPT733/Project/SOCC/raw/gnm_comments.csv"
test = pd.read_csv(test_data)
test = test[['article_id', 'comment_id','comment_text']]
test.head()

/Users/ting/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4,9,10,11,13,14,16,17,18,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,article_id,comment_id,comment_text
0,10012655,33750cc126314c84b4babae99e97b347,I think the program needs some work and probab...
1,10012655,4ce8f60b0ddd442c8a3ac70c15feb954,This is just 'offshoring' in reverse as well a...
2,10012655,75900e8bd92c451491729551878a166d,How do you spell exploitation? This is a disgu...
3,10012655,ac49765f024640ae93e0913cdfbb4d48,TFWs have a place in the economy. Most Canadia...
4,10012655,c5b63fd3000e4306960411384e2999b2,Why should temporary workers get paid more?? H...


In [20]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

p_stemmer = PorterStemmer()

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
#     words = [p_stemmer.stem(w) for w in words]
    #
    # 5. Return a list of words
    return(words)

In [21]:
import nltk
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [22]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/ting/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a U

Parsing sentences from unlabeled set


/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like r

In [23]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
# model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-04-13 09:15:02,271 : INFO : 'pattern' package not found; tag filters are not available for English
2018-04-13 09:15:02,284 : INFO : collecting all words and their counts
2018-04-13 09:15:02,286 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-13 09:15:02,335 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-04-13 09:15:02,411 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-04-13 09:15:02,462 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2018-04-13 09:15:02,522 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-04-13 09:15:02,585 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-04-13 09:15:02,639 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-04-13 09:15:02,695 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-04-13 09:15:02,743 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-04-13 09:15:02,790 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-04-13 09:15:02,843 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2018-04-13 09:15:02,890 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2018-04-13 09:15:02,939 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

2018-04-13 09:15:06,191 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2018-04-13 09:15:06,243 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-04-13 09:15:06,284 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-04-13 09:15:06,331 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-04-13 09:15:06,360 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2018-04-13 09:15:06,361 : INFO : Loading a fresh vocabulary
2018-04-13 09:15:06,430 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-04-13 09:15:06,431 : INFO : min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2018-04-13 09:15:06,478 : INFO : deleting the raw counts dictionary of 123504 items
2018-04-13 09:15:06,482 : INFO : sample=0.001 d

2018-04-13 09:15:55,738 : INFO : EPOCH 4 - PROGRESS: at 7.60% examples, 961765 words/s, in_qsize 7, out_qsize 0
2018-04-13 09:15:56,742 : INFO : EPOCH 4 - PROGRESS: at 15.17% examples, 958091 words/s, in_qsize 7, out_qsize 0
2018-04-13 09:15:57,742 : INFO : EPOCH 4 - PROGRESS: at 22.82% examples, 960600 words/s, in_qsize 7, out_qsize 0
2018-04-13 09:15:58,744 : INFO : EPOCH 4 - PROGRESS: at 30.44% examples, 963134 words/s, in_qsize 7, out_qsize 0
2018-04-13 09:15:59,751 : INFO : EPOCH 4 - PROGRESS: at 37.84% examples, 956327 words/s, in_qsize 7, out_qsize 0
2018-04-13 09:16:00,751 : INFO : EPOCH 4 - PROGRESS: at 45.29% examples, 955450 words/s, in_qsize 7, out_qsize 0
2018-04-13 09:16:01,753 : INFO : EPOCH 4 - PROGRESS: at 52.60% examples, 952596 words/s, in_qsize 7, out_qsize 0
2018-04-13 09:16:02,757 : INFO : EPOCH 4 - PROGRESS: at 59.78% examples, 949224 words/s, in_qsize 7, out_qsize 0
2018-04-13 09:16:03,758 : INFO : EPOCH 4 - PROGRESS: at 67.23% examples, 949292 words/s, in_qsize

In [3]:
# model.doesnt_match("man woman child kitchen".split())

In [4]:
# model.most_similar("awful")

### From Words To Paragraphs, Attempt 1: Vector Averaging

In [36]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [2]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [38]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["comment_text"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/ting/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Review 0 of 25000


/Users/ting/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews


/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://www.youtube.com/watch?v=1qKps7uG6eM" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://5ptsalt.files.wordpress.com/2012/06/sycophant_thumb.jpg?w=766&h=560" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://www.facebook.com/AlternativeRevenueToolsForMetrolinxAndTheBigMove/info" looks like a URL. Beautiful Soup is not an HTTP client. You 

...

/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://theintercept.com/2016/11/09/democrats-trump-and-the-ongoing-dangerous-refusal-to-learn-the-lesson-of-brexit/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://www.washingtonpost.com/news/morning-mix/wp/2016/11/11/video-shows-group-beating-man-in-chicago-yelling-you-voted-trump-and-dont-vote-trump/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/ting/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarn

Review 0 of 663173


/Users/ting/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


Review 1000 of 663173
Review 2000 of 663173
Review 3000 of 663173
Review 4000 of 663173
Review 5000 of 663173
Review 6000 of 663173
Review 7000 of 663173
Review 8000 of 663173
Review 9000 of 663173
Review 10000 of 663173
Review 11000 of 663173
Review 12000 of 663173
Review 13000 of 663173
Review 14000 of 663173
Review 15000 of 663173
Review 16000 of 663173
Review 17000 of 663173
Review 18000 of 663173
Review 19000 of 663173
Review 20000 of 663173
Review 21000 of 663173
Review 22000 of 663173
Review 23000 of 663173
Review 24000 of 663173
Review 25000 of 663173
Review 26000 of 663173
Review 27000 of 663173
Review 28000 of 663173
Review 29000 of 663173
Review 30000 of 663173
Review 31000 of 663173
Review 32000 of 663173
Review 33000 of 663173
Review 34000 of 663173
Review 35000 of 663173
Review 36000 of 663173
Review 37000 of 663173
Review 38000 of 663173
Review 39000 of 663173
Review 40000 of 663173
Review 41000 of 663173
Review 42000 of 663173
Review 43000 of 663173
Review 44000 of 6631

Review 347000 of 663173
Review 348000 of 663173
Review 349000 of 663173
Review 350000 of 663173
Review 351000 of 663173
Review 352000 of 663173
Review 353000 of 663173
Review 354000 of 663173
Review 355000 of 663173
Review 356000 of 663173
Review 357000 of 663173
Review 358000 of 663173
Review 359000 of 663173
Review 360000 of 663173
Review 361000 of 663173
Review 362000 of 663173
Review 363000 of 663173
Review 364000 of 663173
Review 365000 of 663173
Review 366000 of 663173
Review 367000 of 663173
Review 368000 of 663173
Review 369000 of 663173
Review 370000 of 663173
Review 371000 of 663173
Review 372000 of 663173
Review 373000 of 663173
Review 374000 of 663173
Review 375000 of 663173
Review 376000 of 663173
Review 377000 of 663173
Review 378000 of 663173
Review 379000 of 663173
Review 380000 of 663173
Review 381000 of 663173
Review 382000 of 663173
Review 383000 of 663173
Review 384000 of 663173
Review 385000 of 663173
Review 386000 of 663173
Review 387000 of 663173
Review 388000 of

In [44]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# # Test & extract results 
predition = forest.predict(np.nan_to_num(testDataVecs))

# # Write the test results 
# output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
# output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

# test_data_features = test_data_features.toarray()
# predition = forest.predict(test_data_features)

# predition = np.asarray(predition)
# test['sentiment'] = predition
# test.to_csv('comment_w_sentiment.csv')

Fitting a random forest to labeled training data...


In [45]:
test['sentiment'] = predition

In [46]:
test.to_csv('comment_w_sentiment_avg_vector.csv')

### From Words To Paragraphs, Attempt 2: Clustering

In [24]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time
# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] // 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

/Users/ting/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  import sys


Time taken for K Means clustering:  682.3323299884796 seconds.


In [26]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [27]:
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print ("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print (words)


Cluster 0
['lucas', 'ken', 'stanley', 'cameron', 'burton', 'sidney', 'leo', 'cinematographer', 'directs', 'composer', 'olivier', 'samuel', 'hal', 'branagh', 'mann', 'kenneth', 'newman', 'lester', 'lang', 'otto']

Cluster 1
['useless', 'worthless', 'unwatchable']

Cluster 2
['bambi', 'prequel', 'ringu', 'vastly', 'highlander', 'hellraiser', 'bakshi']

Cluster 3
['polish', 'similarly', 'les', 'polanski', 'bunuel', 'crumb', 'nikita', 'influences', 'ki', 'biopic', 'filmography', 'di', 'kazan', 'miyazaki', 'quintessential', 'browning', 'zhang', 'chabrol', 'akira', 'stylistic', 'des', 'besson', 'auteur', 'godard', 'braveheart', 'scorcese', 'antonioni', 'gilliam', 'renowned', 'canon', 'fran', 'boorman', 'opus', 'araki', 'shawshank', 'solondz', 'werner', 'herzog', 'kitano', 'visconti', 'tod', 'fassbinder', 'bu', 'realist', 'europa', 'mulholland', 'ferrara', 'lenzi', 'vertigo', 'truffaut', 'angelopoulos', 'almodovar', 'ozon', 'rohmer', 'depalma', 'loach', 'gregg', 'duk', 'goldsworthy', 'pasoli

In [29]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [ ]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["sentiment"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["sentiment"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1